# VAE 구조

![](https://user-images.githubusercontent.com/24144491/50323466-18d03700-051d-11e9-82ed-afb1b6e2666a.png)

이 구조에서 input 그림이 있을 때 어떤 의미를 가진 구조를 거쳐 output이 나오게 되는지 3 단계로 나누어 살펴볼 것이다.

⏳ **단계별 과정**
1. input: x –> 𝑞_∅ (𝑥)–> 𝜇_𝑖,𝜎_𝑖
2. 𝜇_𝑖, 𝜎_𝑖, 𝜖_𝑖 –> 𝑧_𝑖
3. 𝑧_𝑖 –> 𝑔_𝜃 (𝑧_𝑖) –> 𝑝_𝑖 : output

## sudo of implementations of a VAE
z_mean, z_log_variance = encoder(input_img)

z = z_mean + exp(z_log_variance) * epsilon

reconstructed_img = decode(z)

model = Model(input_img, reconstructed_img)

In [ ]:
# VAE encoder network

# import libraries

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


import keras
from keras import layers
from keras import backend as K
from keras.models import Model

# 1. Encoder
input: x –> 𝑞_∅ (𝑥)–> 𝜇_𝑖,𝜎_𝑖

![](https://user-images.githubusercontent.com/24144491/50323467-1968cd80-051d-11e9-932c-a9b9ef91de58.png)

* Input shape(x) : (28,28,1)
* 𝑞_∅ (𝑥) 는 encoder 함수인데, x가 주어졌을때(given) z값의 분포의 평균과 분산을 아웃풋으로 내는 함수이다.
> Encoder 함수의 output은 latent variable의 분포의 𝜇 와 𝜎 를 내고, 이 output값을 표현하는 확률밀도함수를 생각해볼 수 있다.

In [ ]:
img_shape = (28,28, 1)
batch_size = 16
latent_dim = 2

encoder_input = keras.Input(shape = img_shape, name='Encoder_Input')
x = encoder_input

x = layers.Conv2D(32, 3, padding = 'same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding = 'same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding = 'same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding = 'same', activation='relu')(x)

# return tuple of integers of shape of x
# (14, 14, 64)
shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x) # (None, 12544)
x = layers.Dense(32, activation='relu')(x) # (None, 32)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# 2. Reparameterization Trick (Sampling)
𝜇_𝑖, 𝜎_𝑖, 𝜖_𝑖 –> 𝑧_𝑖

![](https://user-images.githubusercontent.com/24144491/50323468-1968cd80-051d-11e9-8031-ff19c4eaf7c8.png)

sampling 과정이 없다면 encoder 결과에서 나온 값에 대한 decoder역시 한 값만 가지게 된다. 따라서 우리는 필연적으로 그 데이터의 확률분포와 같은 분포에서 하나를 뽑는 sampling을 해야한다. 하지만 그냥 sampling 한다면 sampling 한 값들을 backpropagation 할 수 없다.이를 해결하기 위해 reparmeterization trick을 사용한다고 한다. 

정규분포에서 z1를 샘플링하는 것이나, 입실론을 정규분포(자세히는 N(0,1))에서 샘플링하고 그 값을 분산과 곱하고 평균을 더해 z2를 만들거나 두 z1,z2 는 같은 분포를 가진다고 한다.

그래서 **코드에서 epsilon을 먼저 정규분포에서 random하게 뽑고, 그 epsilon을 exp(z_log_var)과 곱하고 z_mean을 더한다. 그렇게 형성된 값이 z가 된다.**

>latent variable에서 sample된 z라는 value (= decoder input)가 만들어진다.

In [ ]:
# latent_space_sampling function

def sampling(args):
  z_mean, z_log_var = args
  # 정규분포로부터의 난수값을 반환
  epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
  return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# 3. Decoder
𝑧_𝑖 –> 𝑔_𝜃 (𝑧_𝑖) –> 𝑝_𝑖 : output
![](https://user-images.githubusercontent.com/24144491/50323470-1968cd80-051d-11e9-90b1-a2e69ddbcbdd.png)
z 값을 g 함수(decoder)에 넣고 deconv(Conv2DTranspose)를 해 원래 이미지 사이즈의 아웃풋 z_decoded가 나오게 된다. 이때 p_data(x)의 분포를 Bernoulli 로 가정했으므로 output 값은 0~1 사이 값을 가져야하고, 이를 위해 마지막 activatino function이 sigmoid로 설정되는 것이다.

In [ ]:
# VAE decoder network
# latent space의 포인트를 이미지로 맵핑

decoder_input = layers.Input(K.int_shape(z)[1:]) # 잠재벡터의 차원(latent_dim) 과 같다
x = layers.Dense(np.prod(shape_before_flattening), activation = 'relu')(decoder_input) # np.prod(): 각 배열 요소를 곱함
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2,2))(x)
x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)

decoder_output = x
decoder = Model(inputs = decoder_input, outputs = decoder_output, name="Decoder")
z_decoded = decoder(z)

In [ ]:
z_decoded.shape

TensorShape([None, None, None, 1])

In [ ]:
decoder.summary()

Model: "Decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 12544)             37632     
_________________________________________________________________
reshape_11 (Reshape)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 28, 28, 1)         289       
Total params: 56,385
Trainable params: 56,385
Non-trainable params: 0
_________________________________________________________________


# VAE 학습
## Loss Function 이해

Loss 는 크게 총 2가지 부분이 있다.

![](https://user-images.githubusercontent.com/24144491/50323472-1a016400-051d-11e9-86b7-d8bf6a1a880f.png)

* Reconstruction Loss(code에서는 xent_loss)
* Regularization Loss(code에서는 kl_loss)

**Reconstruction Loss : X 와 New X와의 관계**

디코더 부분의 pdf는 Bernoulli 분포를 따른다고 가정했으므로 그 둘간의 cross entropy를 구한다.

**Regularization Loss : X의 분포와 근사한 분포의 차이**

X가 원래 가지는 분포와 동일한 분포를 가지게 학습하게 하기위해 true 분포를 approximate 한 함수의 분포에 대한 loss term이 Regularization Loss다. 이때 loss는 true pdf 와 approximated pdf간의 D_kl(두 확률분포의 차이(거리))을 계산한다.




In [ ]:
# Custom layer used to compute the VAE loss

class CustomVariationalLayer(keras.layers.Layer):

  def vae_loss(self, x, z_decoded):
    x = K.flatten(x)
    print(K.shape(x))
    z_decoded = K.flatten(z_decoded)
    print(K.shape(z_decoded))
    xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
    kl_loss = -5e-4*K.mean(1+z_log_var-K.square(z_mean)-K.exp(z_log_var), axis=-1)
    return K.mean(xent_loss + kl_loss)

  def call(self, inputs):
    x = inputs[0]
    z_decoded = inputs[1]
    loss = self.vae_loss(x,z_decoded)
    self.add_loss(loss, inputs=inputs)
    return x

y = CustomVariationalLayer()([encoder_input, z_decoded]) # check
y

Tensor("custom_variational_layer_8/Shape:0", shape=(1,), dtype=int32)
Tensor("custom_variational_layer_8/Shape_1:0", shape=(1,), dtype=int32)


<tf.Tensor 'custom_variational_layer_8/Identity:0' shape=(None, 28, 28, 1) dtype=float32>

In [ ]:
# Training the VAE

from keras.datasets import mnist

vae = Model(encoder_input, y)
vae.compile(optimizer='rmsprop',loss=None) # Model.compile(loptimizer, loss, metrics)
vae.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 28, 28, 32)   320         Encoder_Input[0][0]              
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 14, 14, 64)   18496       conv2d_51[0][0]                  
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 14, 14, 64)   36928       conv2d_52[0][0]                  
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training_utils.py:819: UserWarning: Output custom_variational_layer_8 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to custom_variational_layer_8.
  'be expecting any data to be passed to {0}.'.format(name))


In [ ]:
y.shape

TensorShape([None, 28, 28, 1])

# MNIST data에 적용

In [ ]:
(x_train,_),(x_test,y_test) = mnist.load_data()

x_train = x_train.astype('float32') /255.
x_train = x_train.reshape(x_train.shape+(1,)) # check
x_test = x_test.astype('float32') /255.
x_test = x_test.reshape(x_test.shape+(1,))

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
vae.fit(x=x_train, y=None, shuffle=True, epochs=10, batch_size=batch_size,validation_data=(x_test,None))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
35008/60000 [================>.............] - ETA: 2:27 - loss: 0.1918

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

n=15
digit_size = 28
figure = np.zeros((digit_size*n, digit_size*n))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
  for j, xi in enumerate(grid_y):
    z_sample = np.array([[xi, yi]])
    z_sample = np.tile(z_sample,batch_size).reshape(batch_size, 2)
    x_decoded = decoder.predict(z_sample, batch_size = batch_size)
    digit = x_decoded[0].reshape(digit_size, digit_size)
    figure[i * digit_size : (i+1)*digit_size, j*digit_size : (j+1)*digit_size] = digit

plt.figure(figsize=(10,10))
plt.imshow(figure, cmap = 'Greys_r')
plt.show()

![](https://user-images.githubusercontent.com/24144491/50323704-289c4b00-051e-11e9-8dad-f32bad3c0583.PNG)

(z1,z2)에서 z1 20개(0.05~0.95), z2 20개, 총 400개의 순서쌍의 xi,yi에서 sample을 뽑아 시각화한것이 오른쪽 그림이다.

참고[https://taeu.github.io/paper/deeplearning-paper-vae/]